# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = False

In [4]:
PREDICTION_METHOD = "Gene-based"

In [5]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [6]:
# OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
# display(OUTPUT_DATA_DIR)
# OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
INPUT_DATA_DIR = Path(
    OUTPUT_DIR,
    "data",
    "raw",
)
display(INPUT_DATA_DIR)
INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
display(gold_standard.shape)

(998, 3)

In [11]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Raw data

In [13]:
input_file = Path(INPUT_DATA_DIR, "lincs-data.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/lincs-data.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
display(lincs_projection.shape)

(7120, 1170)

In [16]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f
    for f in INPUT_DATA_DIR.glob("*.pkl")
    if f.name.startswith("spredixcan-")
    #     if f.name.startswith(("smultixcan-", "spredixcan-"))
]

In [19]:
display(len(phenomexcan_input_file_list))

50

In [20]:
pd.read_pickle(phenomexcan_input_file_list[10]).head()

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000457,-0.000239,-0.000099,-0.000077,-0.000053,-0.000076,0.000005,-0.000045,-0.000166,-0.000120,-0.000097,...,-0.000062,0.000049,2.424838e-05,0.000090,-0.000002,0.000039,0.000032,-0.000097,0.000171,-0.000037
ENSG00000000460,-0.000151,0.000048,0.000030,0.000142,-0.000194,-0.000101,-0.000067,0.000038,0.000135,-0.000009,...,0.000011,0.000003,-9.160683e-05,-0.000031,-0.000055,0.000155,0.000010,-0.000117,-0.000012,-0.000122
ENSG00000000938,0.000171,0.000019,-0.000027,-0.000052,0.000128,-0.000011,0.000012,0.000103,0.000009,-0.000168,...,-0.000043,-0.000017,7.387057e-07,0.000082,-0.000061,-0.000064,0.000055,-0.000028,0.000094,0.000017
ENSG00000000971,0.000198,-0.000003,-0.000008,-0.000052,0.000079,-0.000041,0.000020,0.000132,0.000122,-0.000014,...,0.000293,-0.000075,-9.702798e-05,-0.000027,-0.000025,-0.000090,0.000096,0.000126,0.000117,-0.000002
ENSG00000001036,-0.000113,0.000070,-0.000099,-0.000064,-0.000079,-0.000101,0.000078,0.000055,0.000006,-0.000026,...,-0.000158,0.000030,3.265357e-05,0.000058,0.000085,0.000044,-0.000013,0.000207,-0.000010,-0.000103


# Predict drug-disease associations

In [21]:
from drug_disease import (
#     predict_dotprod,
    predict_dotprod_neg,
#     predict_pearson,
#     predict_pearson_neg,
#     predict_spearman,
#     predict_spearman_neg,
)

In [22]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)

    # get common genes with lincs
    common_genes = phenomexcan_projection.index.intersection(lincs_projection.index)
    phenomexcan_projection = phenomexcan_projection.loc[common_genes]
    lincs_projection = lincs_projection.loc[common_genes]

    print(f"  shape: {phenomexcan_projection.shape}")

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
        n_top_conditions=50,
    )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
        n_top_conditions=100,
    )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
        n_top_conditions=250,
    )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
        n_top_conditions=500,
    )

    #     predict_dotprod(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_pearson(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_pearson_neg(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_spearman(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_spearman_neg(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    print("\n")

spredixcan-mashr-zscores-Adipose_Subcutaneous-data.pkl
  shape: (5734, 4091)
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run


spredixcan-mashr-zscores-Adipose_Visceral_Omentum-data.pkl
  shape: (5082, 4091)
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run


spredixcan-mashr-zscores-Adrenal_Gland-data.pkl
  shape: (4405, 4091)
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run


spredixcan-mashr-zscores-Artery_Aorta-data.pkl
  shape: (4106, 4091)
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run
  predicting... already run


spredixcan-mashr-zscores-Artery_Coronary-data.pkl
  shape: (3766, 4091)
  predicting... already run
  predicting... already run
  predi

KeyboardInterrupt: 